##### Copyright 2019 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Cómo examinar el gráfico de TensorFlow

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/tensorboard/graphs"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/tensorboard/graphs.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/tensorboard/graphs.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fuente en GitHub</a>
</td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/tensorboard/graphs.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar el bloc de notas</a></td>
</table>

## Descripción general

El **Panel de control de Gráficos de TensorBoard** es una potente herramienta para examinar su modelo de TensorFlow. Puede ver rápidamente un gráfico conceptual de la estructura de su modelo y asegurarse de que coincide con su diseño previsto. También puede ver un gráfico a nivel operacional para comprender cómo entiende TensorFlow su programa. Examinar el gráfico a nivel operacional puede darle una idea de cómo cambiar su modelo. Por ejemplo, puede rediseñar su modelo si el entrenamiento está progresando más lentamente de lo esperado.

Este tutorial presenta una rápida visión general de cómo generar datos de diagnóstico de gráficos y visualizarlos en el panel de control de gráficos de TensorBoard. Definirá y entrenará un modelo secuencial simple de Keras para el conjunto de datos Fashion-MNIST y aprenderá a registrar y examinar los gráficos de su modelo. También utilizará una API de seguimiento para generar datos de los gráficos de las funciones creadas utilizando la nueva anotación `tf.function`.

## Preparación

In [2]:
# Load the TensorBoard notebook extension.
%load_ext tensorboard

In [3]:
from datetime import datetime
from packaging import version

import tensorflow as tf
from tensorflow import keras

print("TensorFlow version: ", tf.__version__)
assert version.parse(tf.__version__).release[0] >= 2, \
    "This notebook requires TensorFlow 2.0 or above."

TensorFlow version:  2.2.0


In [4]:
import tensorboard
tensorboard.__version__

'2.2.1'

In [5]:
# Clear any logs from previous runs
!rm -rf ./logs/ 

## Definir un modelo de Keras

En este ejemplo, el clasificador es un modelo secuencial simple de cuatro capas.

In [6]:
# Define the model.
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])

Descargue y prepare los datos de entrenamiento.

In [7]:
(train_images, train_labels), _ = keras.datasets.fashion_mnist.load_data()
train_images = train_images / 255.0

## Entrene el modelo y registre los datos

Antes del entrenamiento, defina la retrollamada de [Keras TensorBoard](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/TensorBoard), especificando el directorio de registro. Al pasar esta retrollamada a Model.fit(), se asegura de que los datos del gráfico se registran para su visualización en TensorBoard.

In [8]:
# Define the Keras TensorBoard callback.
logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

# Train the model.
model.fit(
    train_images,
    train_labels, 
    batch_size=64,
    epochs=5, 
    callbacks=[tensorboard_callback])

Epoch 1/5
938/938 [==============================] - 2s 2ms/step - loss: 0.6955 - accuracy: 0.7618
Epoch 2/5
938/938 [==============================] - 2s 2ms/step - loss: 0.4877 - accuracy: 0.8296
Epoch 3/5
938/938 [==============================] - 2s 2ms/step - loss: 0.4458 - accuracy: 0.8414
Epoch 4/5
938/938 [==============================] - 2s 2ms/step - loss: 0.4246 - accuracy: 0.8476
Epoch 5/5
938/938 [==============================] - 2s 2ms/step - loss: 0.4117 - accuracy: 0.8508


## Gráfico a nivel operacional

Inicie TensorBoard y espere unos segundos a que se cargue la interfaz de usuario. Seleccione el panel de control de Gráficos pulsando "Gráficos" en la parte superior. 

In [ ]:
%tensorboard --logdir logs

También puede utilizar opcionalmente TensorBoard.dev para crear un experimento alojado y que se pueda compartir. 

In [ ]:
!tensorboard dev upload \
  --logdir logs \
  --name "Sample op-level graph" \
  --one_shot

De forma predeterminada, TensorBoard muestra el gráfico **nivel de operación**. (A la izquierda, puede ver la etiqueta "Predeterminada" seleccionada.) Observe que el gráfico está invertido; los datos fluyen de abajo hacia arriba, por lo que está al revés en comparación con el código. Sin embargo, puede ver que el gráfico se asemeja mucho a la definición del modelo de Keras, con aristas adicionales a otros nodos de cálculo.

Los gráficos con frecuencia son muy grandes, por lo que puede manipular la visualización del gráfico:

- Desplácese para hacer **zoom** hacia dentro y hacia fuera
- Arrastre a **panear**
- Al hacer doble clic se activa **la expansión de nodos** (un nodo puede ser contenedor de otros nodos)

También puede ver los metadatos haciendo clic en un nodo. Esto le permite ver las entradas, salidas, formas y otros detalles.


<!-- <img class="tfo-display-only-on-site" src="https://github.com/tensorflow/tensorboard/blob/master/docs/images/graphs_computation.png?raw=1"/> -->

<!-- <img class="tfo-display-only-on-site" src="https://github.com/tensorflow/tensorboard/blob/master/docs/images/graphs_computation_detail.png?raw=1"/> -->

## Gráfico conceptual

Además del gráfico de ejecución, TensorBoard también muestra un **gráfico conceptual**. Esta es una vista sólo del modelo de Keras. Esto puede ser útil si está reutilizando un modelo guardado y desea examinar o validar su estructura.

Para ver el gráfico conceptual, seleccione la etiqueta "keras". En este ejemplo, verá un nodo **Secuencial** colapsado. Haga doble clic en el nodo para ver la estructura del modelo:

<!-- <img class="tfo-display-only-on-site" src="https://github.com/tensorflow/tensorboard/blob/master/docs/images/graphs_tag_selection.png?raw=1"/> --> <br/>
<!-- <img class="tfo-display-only-on-site" src="https://github.com/tensorflow/tensorboard/blob/master/docs/images/graphs_conceptual.png?raw=1"/> -->

## Gráficos de tf.functions

Los ejemplos hasta ahora han descrito gráficos de modelos de Keras, donde los gráficos se crearon definiendo capas de Keras y llamando a Model.fit().

Puede encontrarse con una situación en la que necesite utilizar la anotación `tf.function` para ["autograph"](https://www.tensorflow.org/guide/function), es decir, transformar, una función de cálculo de Python en un gráfico de TensorFlow con alto rendimiento. Para estas situaciones, se utiliza la **API TensorFlow Summary Trace** para registrar las funciones autografiadas para su visualización en TensorBoard.

Para utilizar la API de rastreo de resumen:

- Defina y anote una función con `tf.function`
- Utilice `tf.summary.trace_on()` inmediatamente antes de su sitio de llamada a la función.
- Agregue información al perfil (memoria, tiempo de CPU) al gráfico pasando `profiler=True`
- Con un escritor de archivos de resumen, llame a `tf.summary.trace_export()` para guardar los datos del registro.

A continuación, puede utilizar TensorBoard para ver cómo se comporta su función.


In [10]:
# The function to be traced.
@tf.function
def my_func(x, y):
  # A simple hand-rolled layer.
  return tf.nn.relu(tf.matmul(x, y))

# Set up logging.
stamp = datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = 'logs/func/%s' % stamp
writer = tf.summary.create_file_writer(logdir)

# Sample data for your function.
x = tf.random.uniform((3, 3))
y = tf.random.uniform((3, 3))

# Bracket the function call with
# tf.summary.trace_on() and tf.summary.trace_export().
tf.summary.trace_on(graph=True, profiler=True)
# Call only one tf.function when tracing.
z = my_func(x, y)
with writer.as_default():
  tf.summary.trace_export(
      name="my_func_trace",
      step=0,
      profiler_outdir=logdir)

In [ ]:
%tensorboard --logdir logs/func

<!-- <img class="tfo-display-only-on-site" src="https://github.com/tensorflow/tensorboard/blob/master/docs/images/graphs_autograph.png?raw=1"/> -->

Ahora puede ver la estructura de su función tal y como la entiende TensorBoard. Haga clic en el radiobotón "Perfil" para ver las estadísticas del CPU y la memoria.